In [1]:
import sys
# sys.path.append('../../')

from os import listdir

from hdf5storage import loadmat, savemat 
import numpy as np 
from matplotlib import pyplot as plt

from scipy import signal 
# from hnlpy.timeop import makefiltersos
# from hnlpy.timeop import narrowfilter 
from scipy.signal import sosfiltfilt
from scipy.signal import hilbert
from scipy.signal import savgol_filter
from scipy.stats import sem
from scipy import stats
from scipy.stats import f_oneway
from numpy.polynomial.polynomial import polyfit, polyval

import networkx as nx


In [2]:
# (Color Scheme) 
red         = [1, 0, 0]
pink        = [1, 0.65, 0.75]
black       = [0, 0, 0]
white       = [1, 1, 1]
blue        = [0, 0, 1]
mediumblue  = [0, 0.4, 0.7]
green       = [0, 1, 0]
darkgreen   = [0, 0.5, 0]
grey        = [0.5, 0.5, 0.5]
yellow      = [1, 1, 0]
deepyellow  = [1, 0.8, 0.2]
gold        = [212/255, 175/255, 55/255]
brown       = [150/255, 75/255, 0]
magenta     = [1, 0, 1] 
cyan        = [0, 1, 1]  
purple      = [0.6, 0.1, 0.9]

# % https://www.mathworks.com/help/matlab/creating_plots/specify-plot-colors.html
matlab_blue     = [0, 0.4470, 0.7410]
matlab_orange   = [0.8500, 0.3250, 0.0980]
matlab_gold     = [0.9290, 0.6940, 0.1250]
matlab_purple   = [0.4940, 0.1840, 0.5560]
matlab_green    = [0.4660, 0.6740, 0.1880]
matlab_cyan     = [0.3010, 0.7450, 0.9330]
matlab_red      = [0.6350, 0.0780, 0.1840]

# combine colors
condicolors = [darkgreen, red, blue, magenta, purple, purple]
dire3colors = [darkgreen, brown, magenta]
syn2colors = [darkgreen, pink]
HNLcolors = [darkgreen, deepyellow, pink]

color_4st=['green','red','blue','black']

In [10]:
# load chan locs for topoplots
dict_loc=loadmat('../Archived/Pcorr_4states_2sestype/chan_locs/xyzlabels.mat')

x=dict_loc['x']
y=dict_loc['y']
z=dict_loc['z']
labels=dict_loc['labels']

# adjust coords for nilearn plots
coords=np.zeros((32,3))
for i in range(32):
    coords[i,:]=[x[i]*700, y[i]*900-15, z[i]*950-15]

#  construct text for networkx labels
labeldict = {}
for i in range(32):
    labeldict[i]=labels[i]

# load pos for networkx plots
pos_mat=loadmat('../Archived/Pcorr_4states_2sestype/chan_locs/pos.mat')
pos=pos_mat['pos']


# construct full edges_list
edges_list=list() # each edge is a two element tuple
for i in range(32):
    for j in range(32):
        if i<j: # use indicies to remove half of edges and colors and the diagonal
            edges_list.append((i,j))

In [3]:
# variable names for plotting
states4names=['Uncoupled','Leading','Following','Mutual']
syn2names=['Synch','Synco']
states3names=['Independent','Unidirectional','Bidirectional']
band_labels = ['Delta','Theta','Alpha','Mu','Beta1','Beta2','Gamma']

In [4]:
outdict=loadmat('Indicies.mat')

# condi4_ind          = outdict['condi4_ind']
# syn_ind             = outdict['syn_ind']
syn2_condi4_ind     = outdict['syn2_condi4_ind']

In [5]:
outdict=loadmat('Corr_eeg144.mat')

eeg_empirical_correlation144x2  = outdict['eeg_empirical_correlation144x2']     
eeg_partial_correlation144x2    = outdict['eeg_partial_correlation144x2']       
intervals144                    = outdict['intervals144']                       
samples144                      = outdict['samples144']                         
session144                      = outdict['session144']                         

In [ ]:
# 3 states pcorr
eeg_pcorr_dgctr_mean2x4x7x32=np.zeros((2,3,7,32))

In [ ]:
# organze eeg_pcorr_dgctr into 2x4x7x32 # ((144,2,7,32)) # degree centrality (does not account for weight)

# mean and sem of degree centralities in 32 channels  
eeg_pcorr_dgctr # 144x2x7x32
eeg_pcorr_dgctr_mean2x4x7x32=np.zeros((2,4,7,32)) # 2 syn types x 4 condi x 7 freq x 32 chan
eeg_pcorr_dgctr_sem2x4x7x32=np.zeros((2,4,7,32)) # 2 syn types x 4 condi x 7 freq x 32 chan

for syn in range(2):
    for freq in range(7):
        # Uncoupled
        mat1=eeg_pcorr_dgctr[syn2_condi4_ind[syn][0],0,freq,:]
        mat2=eeg_pcorr_dgctr[syn2_condi4_ind[syn][0],1,freq,:]
        eeg_pcorr_dgctr_mean2x4x7x32[syn,0,freq,:],eeg_pcorr_dgctr_sem2x4x7x32[syn,0,freq,:]=cal_mean_sem_all_chan(mat1,mat2)


        # Leading for L&R
        mat1=eeg_pcorr_dgctr[syn2_condi4_ind[syn][1],0,freq,:]
        mat2=eeg_pcorr_dgctr[syn2_condi4_ind[syn][2],1,freq,:]
        eeg_pcorr_dgctr_mean2x4x7x32[syn,1,freq,:],eeg_pcorr_dgctr_sem2x4x7x32[syn,1,freq,:]=cal_mean_sem_all_chan(mat1,mat2)

        # Following for L&R
        mat1=eeg_pcorr_dgctr[syn2_condi4_ind[syn][1],1,freq,:]
        mat2=eeg_pcorr_dgctr[syn2_condi4_ind[syn][2],0,freq,:]
        eeg_pcorr_dgctr_mean2x4x7x32[syn,2,freq,:],eeg_pcorr_dgctr_sem2x4x7x32[syn,2,freq,:]=cal_mean_sem_all_chan(mat1,mat2)

        # Mutual
        mat1=eeg_pcorr_dgctr[syn2_condi4_ind[syn][3],0,freq,:]
        mat2=eeg_pcorr_dgctr[syn2_condi4_ind[syn][3],1,freq,:]
        eeg_pcorr_dgctr_mean2x4x7x32[syn,3,freq,:],eeg_pcorr_dgctr_sem2x4x7x32[syn,3,freq,:]=cal_mean_sem_all_chan(mat1,mat2)
